In [2]:
from sklearn.datasets import load_wine

In [3]:
import numpy as np

In [4]:
## 1. Activation function
# sigmoid
# softmax
# relu

## 2. Loss function
# mse
# cross_entropy_error


# 3. differ function
# numerical_gradient  Why? 수치미분, (fxh - fx)/h 

# Layer - One

In [22]:
#1 Activation
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.where(x <= 0, 0,x)

def softmax(x):
    x = x - np.max(x,axis=1).reshape(-1,1)
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [24]:
#2 loss Function
def cross_entropy_error(y,t):
    delta = 1e-5
    return -np.sum(t*np.log(y+delta))/y.shape[0]

In [91]:
#3 differ function
def numerical_gradient(f,x):
    h = 1e-4
    grads = np.zeros_like(x)
    if x.ndim == 2:
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                fx = f(x[i,j])
                tmp_val = x[i,j]
                x[i,j] = tmp_val + h
                fxh = f(x[i,j])
                grads[i,j] = (fxh - fx)/h
                x[i,j] = tmp_val
    else:
        for i in range(x.size):
            fx = f(x[i])
            tmp_val = x[i]
            x[i] = tmp_val + h
            fxh = f(x[i])
            grads[i] = (fxh - fx)/h
            x[i] = tmp_val
    return grads

In [32]:
X = load_wine()['data']

In [35]:
y = load_wine()['target']

In [37]:
t = np.zeros((y.size,np.unique(y).size))
for i in range(t.shape[0]):
    t[i,y[i]] = 1

In [47]:
def predict(x,w):
    return softmax(np.dot(x,w))

In [48]:
w = np.random.randn(13,3)
pred = predict(X,w)

In [49]:
f = lambda W: cross_entropy_error(pred,t) 

In [56]:
dW = numerical_gradient(f,w)

In [62]:
lr = 1e-3
dW = numerical_gradient(f,w)
w = w - lr*dW

In [ ]:
#1 network 설계
#2 predict = X*network
#3 predict result vs true value 오차함수 생성
#4 함수를 미분
#5 미분값을 lr와 곱해서 빼준후 update

In [100]:
class OneLayer:
    def __init__(self,input_size,output_size):
        self.W = {}
        self.W['W1'] = np.random.randn(input_size,output_size)
        self.W['b'] = np.random.randn(output_size)
    
    def predict(self,x):
        W1, b = self.W['W1'], self.W['b']
        pred = softmax(np.dot(x,W1) + b)
        return pred
    
    def loss(self,x,t):
        y = self.predict(x)
        return cross_entropy_error(y,t)
    
    def numerical_gradient(self,x,t):
        y = self.predict(x)
        f = lambda W: cross_entropy_error(y,t)
        grad = {}
        grad['W1'] = numerical_gradient(f,self.W['W1'])
        grad['b'] = numerical_gradient(f, self.W['b'])
        
        return grad
    
    def accuracy(self,x,t):
        y = self.predict(x)
        acc = np.sum(np.argmax(y,axis=1) == np.argmax(t,axis=1))/y.shape[0]
        return acc
    
    def fit(self,x,t,epochs=1000,lr=1e-3,verbos=1):
        for epoch in range(epochs):
            self.W['W1'] = self.W['W1'] - lr*self.numerical_gradient(x,t)['W1']
            self.W['b'] -= lr*self.numerical_gradient(x,t)['b']
            if verbos == 1:
                print("=========== loss ",self.loss(x,t), "======== acc ",self.accuracy(x,t))
        

In [101]:
input_size = X.shape[1]
output_size = t.shape[1]
model = OneLayer(input_size=input_size, output_size=output_size)

In [103]:
model.fit(X,t,verbos=2)